In [1]:
# Imports and Initializations
import carla
import math
import random
import time
import numpy as np
import cv2
import subprocess
import open3d as o3d
import matplotlib

# Create a new client instance
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = bp_lib.find('vehicle.dodge.charger')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

# Start traffic-related processes in the background
traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '20', '--number-of-walkers', '30'])
time.sleep(5)

# Set up traffic manager
traffic_manager = client.get_trafficmanager()
vehicle.set_autopilot(True, traffic_manager.get_port())

# Color maps for overlays
VIRIDIS = np.array(matplotlib.colormaps.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(matplotlib.colormaps.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:, :3]  # Remove alpha channel

camera_init_trans = carla.Transform(carla.Location(x=0.0, y=0.0, z=1.6), carla.Rotation(pitch=0.0))

def set_camera_attributes(camera_bp):
    camera_bp.set_attribute('image_size_x', '800')
    camera_bp.set_attribute('image_size_y', '600')
    camera_bp.set_attribute('fov', '90')

# RGB Camera
rgb_camera_bp = bp_lib.find('sensor.camera.rgb')
set_camera_attributes(rgb_camera_bp)
rgb_camera = world.spawn_actor(rgb_camera_bp, camera_init_trans, attach_to=vehicle)

# Semantic Segmentation Camera
sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation')
set_camera_attributes(sem_camera_bp)
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)

# Instance Segmentation Camera
inst_camera_bp = bp_lib.find('sensor.camera.instance_segmentation')
set_camera_attributes(inst_camera_bp)
inst_camera = world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to=vehicle)

# Depth Camera
depth_camera_bp = bp_lib.find('sensor.camera.depth')
set_camera_attributes(depth_camera_bp)
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle)

# GNSS Sensor
gnss_bp = bp_lib.find('sensor.other.gnss')
gnss_sensor = world.spawn_actor(gnss_bp, carla.Transform(), attach_to=vehicle)

# IMU Sensor
imu_bp = bp_lib.find('sensor.other.imu')
imu_sensor = world.spawn_actor(imu_bp, carla.Transform(), attach_to=vehicle)

# Collision Sensor
collision_bp = bp_lib.find('sensor.other.collision')
collision_sensor = world.spawn_actor(collision_bp, carla.Transform(), attach_to=vehicle)

# Obstacle Sensor – note the adjusted transform
obstacle_bp = bp_lib.find('sensor.other.obstacle')
obstacle_bp.set_attribute('hit_radius', '2')
obstacle_bp.set_attribute('distance', '50')
obstacle_sensor = world.spawn_actor(obstacle_bp, 
                                    carla.Transform(carla.Location(x=2.5, z=1.5)),
                                    attach_to=vehicle)

# ---------------------------
# Initialize image data storage and sensor data dictionary
# ---------------------------
image_w = 800
image_h = 600
sensor_data = {
    'rgb_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'sem_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'inst_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'depth_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    # Extra sensor fields:
    'gnss': [0, 0],
    'imu': {
        'gyro': carla.Vector3D(),
        'accel': carla.Vector3D(),
        'compass': 0
    },
    'collision': False,
    'obstacle': []
}

# ---------------------------
# Define helper functions for overlays and sensor info
# ---------------------------
def add_label(image, label, position=(10, 50), font_scale=0.8, color=(255, 255, 255)):
    labeled_image = image.copy()
    cv2.putText(labeled_image, label, position, cv2.FONT_HERSHEY_SIMPLEX, 
                font_scale, color, 2, cv2.LINE_AA)
    return labeled_image

# Camera image callbacks
def rgb_callback(image, data_dict):
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data_dict['rgb_image_raw'] = img.copy()  # Plain copy without overlays
    data_dict['rgb_image'] = add_label(img, "RGB Camera")

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data_dict['sem_image'] = add_label(img, "Semantic Segmentation")

def inst_callback(image, data_dict):
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data_dict['inst_image'] = add_label(img, "Instance Segmentation")

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data_dict['depth_image'] = add_label(img, "Depth Camera")

def gnss_callback(data, data_dict):
    data_dict['gnss'] = [data.latitude, data.longitude]

def imu_callback(data, data_dict):
    data_dict['imu'] = {
        'gyro': data.gyroscope,
        'accel': data.accelerometer,
        'compass': data.compass
    }

def collision_callback(event, data_dict):
    data_dict['collision'] = True

# For projecting obstacle positions onto the image we need an intrinsic matrix.
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

def get_image_point(loc, K, w2c):
    point = np.array([loc.x, loc.y, loc.z, 1])
    point_camera = np.dot(w2c, point)
    point_camera = [point_camera[1], -point_camera[2], point_camera[0]]
    point_img = np.dot(K, point_camera)
    point_img[0] /= point_img[2]
    point_img[1] /= point_img[2]
    return tuple(map(int, point_img[0:2]))

def obstacle_callback(event, data_dict, camera, K, image_w, image_h):
    if not ('vehicle' in event.other_actor.type_id or 'walker' in event.other_actor.type_id):
        return
    data_dict.setdefault('obstacle', []).append({
        'transform': event.other_actor.type_id,
        'frame': event.frame
    })
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
    obstacle_location = event.other_actor.get_location()
    image_point = get_image_point(obstacle_location, K, world_2_camera)
    if 'rgb_image' not in data_dict or data_dict['rgb_image'] is None:
        return
    if 0 < image_point[0] < image_w and 0 < image_point[1] < image_h:
        cv2.circle(data_dict['rgb_image'], tuple(image_point), 10, (0, 0, 255), 2)

def draw_compass(img, theta):
    compass_center = (700, 100)
    compass_size = 50
    cardinal_directions = [
        ('N', [0, -1]),
        ('E', [1, 0]),
        ('S', [0, 1]),
        ('W', [-1, 0])
    ]
    for label, offset in cardinal_directions:
        cv2.putText(
            img, label,
            (int(compass_center[0] + 1.2 * compass_size * offset[0]), 
             int(compass_center[1] + 1.2 * compass_size * offset[1])),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 100, 200), 2
        )
    compass_point = (
        int(compass_center[0] + compass_size * math.sin(theta)),
        int(compass_center[1] - compass_size * math.cos(theta))
    )
    cv2.line(img, compass_center, compass_point, (0, 140, 255), 3)

# Register sensor callbacks
rgb_camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
inst_camera.listen(lambda image: inst_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))
time.sleep(5)
gnss_sensor.listen(lambda data: gnss_callback(data, sensor_data))
imu_sensor.listen(lambda data: imu_callback(data, sensor_data))
collision_sensor.listen(lambda event: collision_callback(event, sensor_data))
obstacle_sensor.listen(lambda event: obstacle_callback(event, sensor_data, rgb_camera,
                                                         build_projection_matrix(image_w, image_h, 90),
                                                         image_w, image_h))

# LiDAR & Radar setup (not modified here)
lidar_bp = bp_lib.find('sensor.lidar.ray_cast_semantic')
lidar_bp.set_attribute('range', '100.0')
lidar_bp.set_attribute('upper_fov', '15.0')
lidar_bp.set_attribute('lower_fov', '-25.0')
lidar_bp.set_attribute('channels', '64')
lidar_bp.set_attribute('rotation_frequency', '20.0')
lidar_bp.set_attribute('points_per_second', '500000')
lidar_init_trans = carla.Transform(carla.Location(z=2))
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)
time.sleep(5)

radar_bp = bp_lib.find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '30.0')
radar_bp.set_attribute('vertical_fov', '30.0')
radar_bp.set_attribute('points_per_second', '10000')
radar_init_trans = carla.Transform(carla.Location(z=2))
radar = world.spawn_actor(radar_bp, radar_init_trans, attach_to=vehicle)

lidar_pcd = o3d.geometry.PointCloud()
radar_pcd = o3d.geometry.PointCloud()

def get_random_color():
    color = np.random.rand(3)
    while np.linalg.norm(color) < 0.3:
        color = np.random.rand(3)
    return color

def lidar_callback(point_cloud, pcd):
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    if data.shape[0] == 0:
        return
    data = np.reshape(data, (-1, 6))
    points = data[:, :3]
    colors = np.array([get_random_color() for _ in range(points.shape[0])])
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)

def radar_callback(data, pcd):
    radar_data = np.zeros((len(data), 4))
    for i, detection in enumerate(data):
        x = detection.depth * math.cos(detection.altitude) * math.cos(detection.azimuth)
        y = detection.depth * math.cos(detection.altitude) * math.sin(detection.azimuth)
        z = detection.depth * math.sin(detection.altitude)
        radar_data[i, :] = [x, y, z, detection.velocity]
    intensity = np.abs(radar_data[:, -1])
    intensity_safe = np.clip(intensity, 1e-6, None)
    intensity_col = 1.0 - np.log(intensity_safe) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, COOL_RANGE, COOL[:, 0]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 1]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 2])
    ]
    points = radar_data[:, :3]
    points[:, :1] = -points[:, :1]
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(int_color)

lidar.listen(lambda data: lidar_callback(data, lidar_pcd))
radar.listen(lambda data: radar_callback(data, radar_pcd))

cv2.namedWindow('All Cameras', cv2.WINDOW_NORMAL)
cv2.resizeWindow('All Cameras', 1280, 960)
# We are no longer calling cv2.imshow for the duplicate RGB below
# cv2.namedWindow('Duplicate RGB', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('Duplicate RGB', 800, 600)

def preprocess_image(img):
    if img.shape[2] == 4:
        img = img[:, :, :3]
    return img

vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Lidar View', width=960, height=540, left=400, top=270)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True

def add_open3d_axis(vis):
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [0.1, 0.0, 0.0],
        [0.0, 0.1, 0.0],
        [0.0, 0.0, 0.1]
    ]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]
    ]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ]))
    vis.add_geometry(axis)
add_open3d_axis(vis)

lidar_added = False
radar_added = False

# Counter for collision display duration
collision_counter = 20

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# ByteTrack & YOLO Initialization
import os
import torch
import sys

# Append ByteTrack path (adjust the path as needed)
sys.path.append("C:/Users/okeiy/OneDrive - University of Salford/Documents/Sch Notes/Dissertation/Codes/Dataset/ByteTrack-ONNX-Sample-main")
from byte_tracker.tracker.byte_tracker import BYTETracker

# --- Load YOLO Model ---
model_path = r"C:\Users\okeiy\OneDrive - University of Salford\Documents\Sch Notes\Dissertation\Codes\Dataset\Yolo\bdd100k\my_yolov11_model.pt"
model = torch.load(model_path, map_location=torch.device('cuda'))
model.model.eval()  # set model to evaluation mode

# --- Initialize ByteTrack Tracker ---
class TrackerArgs:
    def __init__(self):
        self.track_thresh = 0.5   # detection score threshold for tracking
        self.track_buffer = 30    # frames to keep lost tracks
        self.match_thresh = 0.8   # matching threshold
        self.mot20 = False
args = TrackerArgs()
tracker_byte = BYTETracker(args, frame_rate=30)

# --- Set parameters for ByteTrack processing ---
target_size = 640  # network input size
conf_thresh = 0.4  # detection confidence threshold

# --- Helper: Letterbox Resizing ---
def letterbox(image, new_shape=(640, 640), color=(114, 114, 114)):
    shape = image.shape[:2]  # current shape [height, width]
    ratio = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    new_unpad = (int(round(shape[1] * ratio)), int(round(shape[0] * ratio)))  # (width, height)
    dw = new_shape[1] - new_unpad[0]
    dh = new_shape[0] - new_unpad[1]
    dw /= 2
    dh /= 2
    resized = cv2.resize(image, new_unpad, interpolation=cv2.INTER_LINEAR)
    top = int(round(dh - 0.1))
    bottom = int(round(dh + 0.1))
    left = int(round(dw - 0.1))
    right = int(round(dw + 0.1))
    padded = cv2.copyMakeBorder(resized, top, bottom, left, right,
                                cv2.BORDER_CONSTANT, value=color)
    return padded, ratio, left, top

# --- Helper: Compute IoU between two boxes ---
def compute_iou(box1, box2):
    x_left   = max(box1[0], box2[0])
    y_top    = max(box1[1], box2[1])
    x_right  = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    inter_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = inter_area / float(box1_area + box2_area - inter_area + 1e-6)
    return iou

# --- Define the class names ---
class_names = ["bike", "bus", "car", "motor", "person", "rider", "traffic light", "traffic sign", "train", "truck"]

# --- Initialize ByteTrack output video writer ---
byte_output_dir = r"C:\Users\okeiy\OneDrive - University of Salford\Documents\Sch Notes\Dissertation\Codes\Output"
os.makedirs(byte_output_dir, exist_ok=True)
byte_output_video_path = os.path.join(byte_output_dir, "Carla_Output_3.mp4")
# Assume the original duplicate RGB frame has dimensions image_w x image_h
byte_frame_width = image_w
byte_frame_height = image_h
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
byte_out = cv2.VideoWriter(byte_output_video_path, fourcc, 30, (byte_frame_width, byte_frame_height))

C:\Users\okeiy\AppData\Local\Temp\ipykernel_644\3443818359.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=torch.device('cud

In [3]:
# Main Processing Loop with ByteTrack Integration
frame_count = 0
overall_start_time = time.time()
prev_time = time.time()

while True:
    try:
        # Get the current raw RGB image (without overlays)
        if 'rgb_image_raw' in sensor_data:
            duplicate_rgb = sensor_data['rgb_image_raw'].copy()
        else:
            duplicate_rgb = np.zeros((image_h, image_w, 4), dtype=np.uint8)

        # Process the overlay image (for tiled display)
        rgb_img = sensor_data['rgb_image'].copy()
        font = cv2.FONT_HERSHEY_PLAIN
        fontScale = 0.5
        fontScale_2 = 1.0
        fontColor = (255, 255, 255)
        thickness = 2

        start_y = 500  # starting y coordinate for sensor overlays
        line_spacing = 20

        lat_val = sensor_data['gnss'][0] * 100000
        long_val = sensor_data['gnss'][1] * 100000
        cv2.putText(rgb_img, 'Lat: ' + f"{lat_val:.2f}", (10, start_y), font, fontScale_2, fontColor, thickness, cv2.LINE_AA)
        cv2.putText(rgb_img, 'Long: ' + f"{long_val:.2f}", (10, start_y + line_spacing), font, fontScale_2, fontColor, thickness, cv2.LINE_AA)

        accel = sensor_data['imu']['accel'] - carla.Vector3D(x=0, y=0, z=9.81)
        accel_val = accel.length() * 100
        cv2.putText(rgb_img, 'Accel: ' + f"{accel_val:.2f}", (10, start_y + 2 * line_spacing), font, fontScale_2, fontColor, thickness, cv2.LINE_AA)
        
        gyro_val = sensor_data['imu']['gyro'].length() * 100
        cv2.putText(rgb_img, 'Gyro: ' + f"{gyro_val:.2f}", (10, start_y + 3 * line_spacing), font, fontScale_2, fontColor, thickness, cv2.LINE_AA)

        compass_val = sensor_data['imu']['compass'] * 10
        cv2.putText(rgb_img, 'Compass: ' + f"{compass_val:.2f}", (10, start_y + 4 * line_spacing), font, fontScale_2, fontColor, thickness, cv2.LINE_AA)
        draw_compass(rgb_img, sensor_data['imu']['compass'])

        # Display collision alert if needed
        if sensor_data['collision']:
            collision_counter -= 1
            if collision_counter <= 1:
                sensor_data['collision'] = False
            cv2.putText(rgb_img, 'COLLISION', (250, 300), font, 2, (255, 255, 255), 3, cv2.LINE_AA)
        else:
            collision_counter = 20

        sensor_data['rgb_image'] = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2BGRA)

        # Prepare images for tiled display (drop alpha channel if needed)
        processed_data = {key: preprocess_image(img) for key, img in sensor_data.items() if 'image' in key and key != 'rgb_image_raw'}
        top_row = np.concatenate([processed_data['rgb_image'], processed_data['sem_image']], axis=1)
        bottom_row = np.concatenate([processed_data['depth_image'], processed_data['inst_image']], axis=1)
        tiled = np.concatenate((top_row, bottom_row), axis=0)
        cv2.imshow('All Cameras', tiled)

        # ---------------------------
        # Instead of showing the Duplicate RGB window, feed it to ByteTrack
        # ---------------------------
        # Preprocess the duplicate RGB (remove alpha channel) and ensure contiguous memory layout
        byte_frame = preprocess_image(duplicate_rgb)
        byte_frame = np.ascontiguousarray(byte_frame)

        # --- ByteTrack Processing ---
        padded_frame, ratio, pad_w, pad_h = letterbox(byte_frame, (target_size, target_size))
        img_rgb = cv2.cvtColor(padded_frame, cv2.COLOR_BGR2RGB)
        img_normalized = img_rgb.astype(np.float32) / 255.0
        input_tensor = torch.from_numpy(img_normalized).permute(2, 0, 1).unsqueeze(0)

        with torch.no_grad():
            results = model(input_tensor)
        res = results[0]
        boxes = res.boxes.xyxy.cpu().numpy()      
        scores = res.boxes.conf.cpu().numpy()       
        raw_labels = res.boxes.cls.cpu().numpy()    

        valid_inds = scores >= conf_thresh
        boxes = boxes[valid_inds]
        scores = scores[valid_inds]
        det_labels = raw_labels[valid_inds]
        detections = np.concatenate([boxes, scores.reshape(-1, 1)], axis=1)

        detection_boxes_orig = []
        for box in boxes:
            x1_orig = (box[0] - pad_w) / ratio
            y1_orig = (box[1] - pad_h) / ratio
            x2_orig = (box[2] - pad_w) / ratio
            y2_orig = (box[3] - pad_h) / ratio
            detection_boxes_orig.append([x1_orig, y1_orig, x2_orig, y2_orig])
        detection_boxes_orig = np.array(detection_boxes_orig)

        img_info = (target_size, target_size)
        img_size = (target_size, target_size)
        tracks = tracker_byte.update(detections, img_info, img_size)

        for track in tracks:
            x1, y1, x2, y2 = track.tlbr
            x1_orig = int((x1 - pad_w) / ratio)
            y1_orig = int((y1 - pad_h) / ratio)
            x2_orig = int((x2 - pad_w) / ratio)
            y2_orig = int((y2 - pad_h) / ratio)
            track_id = track.track_id

            track_box_orig = [x1_orig, y1_orig, x2_orig, y2_orig]
            best_iou = 0
            best_label = None
            for i, det_box in enumerate(detection_boxes_orig):
                iou = compute_iou(track_box_orig, det_box)
                if iou > best_iou:
                    best_iou = iou
                    best_label = int(det_labels[i])
            if best_iou < 0.3:
                best_label = -1
            if best_label >= 0 and best_label < len(class_names):
                label_text = class_names[best_label]
            else:
                label_text = "unknown"

            text = f"ID:{track_id} {label_text}"
            cv2.rectangle(byte_frame, (x1_orig, y1_orig), (x2_orig, y2_orig), (0, 255, 0), 2)
            cv2.putText(byte_frame, text, (x1_orig, y1_orig - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        current_time = time.time()
        fps = 1.0 / (current_time - prev_time)
        prev_time = current_time
        cv2.putText(byte_frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

        byte_out.write(byte_frame)
        cv2.imshow("ByteTrack Output", byte_frame)

        # Update Open3D visualizer with LiDAR and Radar point clouds
        if not lidar_added:
            vis.add_geometry(lidar_pcd)
            lidar_added = True
        if not radar_added:
            vis.add_geometry(radar_pcd)
            radar_added = True
        vis.update_geometry(lidar_pcd)
        vis.update_geometry(radar_pcd)
        vis.poll_events()
        vis.update_renderer()

        # Use a slightly longer waitKey to help the OS process GUI events
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

        frame_count += 1
        # Reduce sleep to give more time to GUI events (or remove if necessary)
        time.sleep(0.001)

    except Exception as e:
        print("Error:", e)
        break

overall_end_time = time.time()
total_time = overall_end_time - overall_start_time
avg_fps = frame_count / total_time
print(f"Average FPS: {avg_fps:.2f}")


0: 640x640 5 cars, 1 traffic sign, 18.0ms
Speed: 2.1ms preprocess, 18.0ms inference, 190.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 62.7ms
Speed: 2.0ms preprocess, 62.7ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 cars, 336.4ms
Speed: 2.5ms preprocess, 336.4ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 1061.7ms
Speed: 1.0ms preprocess, 1061.7ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 cars, 15.5ms
Speed: 3.5ms preprocess, 15.5ms inference, 5.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 cars, 396.2ms
Speed: 3.5ms preprocess, 396.2ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 cars, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 cars, 382.7ms
Speed: 2.0ms preprocess, 382.7ms inference, 9.5ms postprocess per image at shape

In [4]:
# Shutdown
cv2.destroyAllWindows()
vis.destroy_window()
byte_out.release()